This notebook contains a regression analysis  to test the influence of RLHF and model size of the Llama model class on the emergence of the optimism bias.

In [1]:
import pandas as pd
from glob import glob
import numpy as np
import statsmodels.api as sm

base_path = '../llm/partial/data/'
llms = glob(base_path + 'llama*/')

merged = pd.DataFrame()
for llm in llms:
    df = pd.read_csv(f'{llm}CM_fit_q0.5.csv')
    df['llm'] = llm.split('/')[-2]
    merged = pd.concat([merged, df], ignore_index=True)

merged = merged[merged['cognitive_model']=='Model_2alpha']

df = merged

In [2]:
df['RLHF'] = np.where(df['llm'].str.contains('chat'), 1, 0) # if RLHF true, then 1 else 0
df['model_size'] = df['llm'].str.replace('-chat','').str.split('-').str[-1].astype(int)
df['optimism'] = df['alpha_pos']-df['alpha_neg']
df[['llm', 'model_size', 'RLHF', 'optimism', 'cognitive_model', 'run', 'alpha_pos', 'alpha_neg',]].sort_values('model_size')

,llm,model_size,RLHF,optimism,cognitive_model,run,alpha_pos,alpha_neg
199,llama-2-7,7,0,0.798374,Model_2alpha,49,0.808209,0.009834
195,llama-2-7,7,0,0.956906,Model_2alpha,45,0.959583,0.002676
196,llama-2-7,7,0,0.880020,Model_2alpha,46,0.881823,0.001803
197,llama-2-7,7,0,0.968492,Model_2alpha,47,0.985557,0.017065
198,llama-2-7,7,0,0.810851,Model_2alpha,48,0.813073,0.002221
...,...,...,...,...,...,...,...,...
79,llama-2-70-chat,70,1,0.866840,Model_2alpha,29,0.868128,0.001289
78,llama-2-70-chat,70,1,0.802929,Model_2alpha,28,0.803946,0.001018
77,llama-2-70-chat,70,1,0.858833,Model_2alpha,27,0.861596,0.002763
87,llama-2-70-chat,70,1,-0.000003,Model_2alpha,37,0.499998,0.500001


In [3]:
# Preparing the data for regression analysis
X = df[['RLHF', 'model_size']]
X = sm.add_constant(X)  # Adding the bias (constant) term
y = df['optimism']

# Conducting the regression analysis
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               optimism   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     26.94
Date:                Sat, 25 May 2024   Prob (F-statistic):           4.54e-11
Time:                        23:24:35   Log-Likelihood:                -62.388
No. Observations:                 200   AIC:                             130.8
Df Residuals:                     197   BIC:                             140.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6905      0.044     15.687      0.000       0.604       0.777
RLHF           0.2108      0.047      4.476      0.000       0.118       0.304
model_size    -0.0043      0.001     -5.818      0.000      -0.006      -0.003
==============================================================================
Omnibus:                       44.292   Durbin-Watson:                   1.678
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               69.606
Skew:                          -1.214   Prob(JB):                     7.68e-16
Kurtosis:                       4.568   Cond. No.                         119.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""